In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Number of experiments to run
n_experiments = 100

# The set of class labels in the sample
labels = ['healthy','ring','troph','schizont']
n_labels = len(labels)

# Define confusion matrix. Rows are ground truth, columns are output. Confusion matrix in this form
# is normalized such that all rows sum to 1 and is indexed in the same order as the 'labels' variable

cm = np.array([[0.97, 0.02, 0.01, 0.0], \
               [0.02, 0.97, 0.01, 0.0], \
               [0.01, 0.01, 0.97, 0.01], \
               [0.0, 0.01, 0.02, 0.97]])

# Ground truth sample composition (sums to 1)
sample_comp = np.array([1, 0.0002, 0.0001, 0.0001])
sample_comp[0] = 1 - np.sum(sample_comp[1:])
assert np.sum(sample_comp) == 1.0


In [3]:
# Define the duration of the imaging experiment, in seconds
t_max_s = 20*60

# Inference time per image. Assume for the moment we're limited by inference time
t_inf_s = 0.03

# Total images processed
n_images = t_max_s/t_inf_s

# Average number of cells per image
n_cells_per_image = 20

# Total cells imaged
n_cells_total = int(n_cells_per_image*n_images)
print(n_cells_total)

800000


In [6]:
# numpy random number generator
rng = np.random.default_rng()

# Ground truth class counts for each experiment
sample_gt = np.zeros((n_experiments, n_labels))

# Classifier's estimate of the sample
est = np.zeros((n_experiments, n_labels))
est_parasitemia = np.zeros((n_experiments, n_labels))

for e in range(n_experiments):
    
    r = rng.random(n_cells_total)
    # Create a random variable with length equal to the total number of cells
    
    # Partition
    p = 0.0
    
    # Partition the class counts in proportion to the random number distribution between 0-1
    for i in range(n_labels):
        sample_gt[e,i] = np.count_nonzero( np.logical_and(np.greater(r,p),np.less(r, np.sum(sample_comp[0:(i+1)]))))
        p += sample_comp[i]
    
    # Compute the sample estimates based on the confusion matrix
    est[e,:] = np.matmul(cm,sample_gt[e])
    
    # Sample composition estimate (parasitemia)
    est_parasitemia[e,:] = 100*est[e,:]/np.sum(est[e, :])
    
# Average over experiments
# Total number of parasites in sample
sample_gt_mean = np.average(sample_gt,0)
mean_est_parasitemia = np.average(est_parasitemia, 0)
std_est_parasitemia = np.std(est_parasitemia, 0)
absolute_parasitemia_error = mean_est_parasitemia - 100*sample_comp
fold_error = np.divide(absolute_parasitemia_error, 100*sample_comp)

# Make dicts for easier association with labels
sample_gt_dict = dict(zip(labels, sample_gt_mean))
abs_error_dict = dict(zip(labels,absolute_parasitemia_error))
fold_error_dict = dict(zip(labels,fold_error))
mean_dict = dict(zip(labels,mean_est_parasitemia))
gt_dict = dict(zip(labels, 100*sample_comp))
std_est_dict = dict(zip(labels, std_est_parasitemia))

In [10]:
print("\nAvg Healthy: {healthy:2.3f} \nAvg Ring: {ring:2.3f}\nAvg Troph: {troph:2.3f}\nAvg Schizont: {schizont:2.3f}".format(**sample_gt_dict))
print("\nGround truth composition: \nHealthy: {healthy:2.3f}% \nRing: {ring:2.3f}%\nTroph: {troph:2.3f}%\nSchizont: {schizont:2.3f}% ".format(**gt_dict))
print("\nMean of the estimates: \nHealthy: {healthy:2.3f}% \nRing: {ring:2.3f}%\nTroph: {troph:2.3f}%\nSchizont: {schizont:2.3f}% ".format(**mean_dict))
print("\nAbsolute error: \nHealthy: {healthy:2.3f}% \nRing: {ring:2.3f}%\nTroph: {troph:2.3f}%\nSchizont: {schizont:2.3f}% ".format(**abs_error_dict))
print("\nFold error: \nHealthy: {healthy:2.3f} \nRing: {ring:2.3f}\nTroph: {troph:2.3f}\nSchizont: {schizont:2.3f}".format(**fold_error_dict))
print("\nStdev across estimates: \nHealthy: {healthy:2.5f} \nRing: {ring:2.5f}\nTroph: {troph:2.5f}\nSchizont: {schizont:2.5f}".format(**std_est_dict))


Avg Healthy: 799680.230 
Avg Ring: 161.310
Avg Troph: 79.750
Avg Schizont: 78.710

Ground truth composition: 
Healthy: 99.960% 
Ring: 0.020%
Troph: 0.010%
Schizont: 0.010% 

Mean of the estimates: 
Healthy: 96.962% 
Ring: 2.019%
Troph: 1.010%
Schizont: 0.010% 

Absolute error: 
Healthy: -2.998% 
Ring: 1.999%
Troph: 1.000%
Schizont: -0.000% 

Fold error: 
Healthy: -0.030 
Ring: 99.943
Troph: 99.957
Schizont: -0.006

Stdev across estimates: 
Healthy: 0.00230 
Ring: 0.00159
Troph: 0.00111
Schizont: 0.00102
